### Import and Setups

In [25]:
from pathlib import Path

import numpy as np

import project_path
from src.mlgraph import MLGraph
from src.commdetect import modularity, scml
from src import surrogates

from IPython.display import clear_output

PROJECT_DIR = Path("..")
DATA_DIR = Path(PROJECT_DIR, "data", "random")

### Load Data

We start with loading data. Since EEG data used in the paper is private, we use
simulated multilayer networks generated randomly. 
 

In particular, there are 5 multilayer networks with 4 layers and there are 64
nodes at each layer, similar to EEG networks analyzed in the paper. The network
are fully connected and edge weights are drawn randomly from a uniform
distribution. They don't have any ground truth community structure; so, these
networks are only used to illustrate how the proposed pipeline can be used.
Results obtained from them are not related to conclusions made in the paper. 

In [6]:
graphs = []
for s in range(5):
    G = MLGraph()
    G.read_from_gml(Path(DATA_DIR, f"net_{s+1}.gml"))
    graphs.append(G)

The pipeline requires multilayer graphs to be
[`MLGraph`](../src/mlgraph/__init__.py) objects, which is a composite class that
aims to extend `igraph` to hold multilayer networks. See class definitions for
details of the class. We used `read_from_gml` method of `MLGraph` class to read
a multilayer graph from a `.gml` file (in this case a zipped GML file).

### Find community structures

Once 5 graphs are loaded, we find community structure of each network separately
using the proposed multilayer modularity. We will fix resolution parameter and 
interlayer scale to 1 and discussion on how their optimal value can be found is
given later.

Maximization of multilayer modularity can be performed as follows:

In [7]:
gamma = 1
omega = 1
n_runs = 10 # number of times to run modularity maximization

comms = [] # We will save community structures of each network 
mods = [] # Also modularity values of found community structures
for s, G in enumerate(graphs):
    A = G.supra_adjacency(weight="weight")
    P = modularity.ml_null_matrix(A, G.layers)
    comms.append(modularity.find_ml_communities(A, P, gamma, omega, n_runs))
    mods = modularity.ml_modularity_value(comms[-1], A, P, gamma, omega)

For each network, we first get its supra-adjacency matrix and null matrix of 
the proposed modularity function. The null matrix is constructed using 
`ml_null_matrix` function defined under 
[`modularity`](../src/commdetect/modularity/) module. These two matrices are 
then given to `find_ml_communities` function, which optimizes the proposed 
multilayer modularity. Modularity maximization is run 10 times (in the paper,
number of runs is 100). We then calculated multilayer modularity values of the 
found community structures. 

### Surrogate data generation

Resolution parameter and interlayer scales are choosen by comparing modularity
values of community structures of the observed multilayer networks to those of 
the surrogate networks. For this, we need to generate surrogate networks from 
observed multilayer network as follows:

In [14]:
surr_nets = []
n_surrogates = 10 # number of surrogates network to generate
for s in range(5):
    surr_nets.append([])
    for n in range(n_surrogates):
        surr_nets[-1].append(surrogates.weight_preserved(
            graphs[s], weight="weight", preserve_layer=True
        ))

    print(f"{(s+1)/5*100:.2f}% is done.")

20.00% is done.
40.00% is done.
60.00% is done.
80.00% is done.
100.00% is done.


`surrogates.weight_preserved` function can be used to generate surrogates. We 
generated 10 surrogates for each network being analyzed. 

We need to find communities of surrogates networks at the same resolution 
parameter and interlayer scale values.

In [15]:
gamma = 1
omega = 1

surr_mods = [] # Also modularity values of found community structures
for s in range(5):
    surr_mods.append([])
    for n in range(n_surrogates): 
        A = surr_nets[s][n].supra_adjacency(weight="weight")
        P = modularity.ml_null_matrix(A, surr_nets[s][n].layers)
        surr_comms = modularity.find_ml_communities(A, P, gamma, omega, 1)
        surr_mods[s].append(
            modularity.ml_modularity_value(surr_comms, A, P, gamma, omega)
        )

Multilayer modularity values of these community structure are then used to
determine optimal resolution parameter and interlayer scale. We will skip this
step, as we fixed their values. However, community structures of observed and
surrogate networks can be found for different values as described in the paper.

### Group community structure

Given community structures of multiple related multilayer networks, we can find
a group community structure, summarizing their shared community structure. For 
this, we first find co-clustering matrix for each multilayer network. These 
co-clustering matrices are then used as layer of multiplex network. SCML is then 
applied to the constructed multiplex network to find group community structure.

In [29]:
mx = []
n_comms = []
for s in range(5):
    mx.append(modularity.coclustering_matrix(comms[s]))
    mx[-1] = mx[-1]/np.max(mx[-1])
    mx[-1][np.diag_indices_from(mx[-1])] = 0

    # SCML requires number of communities, which will be determined as the 
    # average of number of communities in the community structure used to 
    # construct co-clustering matrices
    for i in range(10):
        n_comms.append(len(np.unique(comms[s][:, i])))

n_comms = round(np.mean(n_comms))

Finally, we find the group community structure.

In [31]:
c_group = scml.run(mx, n_comms)